In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

In [2]:
# Designed to be used with the zip file directly. Feel free to extract and not run this cell
# !unzip 'train.zip'

In [3]:
# Generate Dataloaders

np.random.seed(321)
train_dir = 'train'
train_files = os.listdir(train_dir)
all_dogs = []
all_cats = []
for i in train_files:
    if('dog' in i):
        all_dogs.append('train/' + i)
    elif('cat' in i):
        all_cats.append('train/' + i)
ind_dog = []
ind_cat = []
names_dog_test = []
names_cat_test = []

for i in range(4000):
    r = np.random.randint(0,12500)
    if r not in ind_dog: 
        ind_dog.append(r)
    if(len(np.unique(np.array(ind_dog))) == 2500):
        break

for i in range(4000):
    r = np.random.randint(0,12500)
    if r not in ind_cat: 
        ind_cat.append(r)
    if(len(np.unique(np.array(ind_cat))) == 2500):
        break
        
for i in ind_dog:
    names_dog_test.append(all_dogs[i])
for i in ind_cat:
    names_cat_test.append(all_cats[i])
    
names_dog_train  = []
names_cat_train = []
cc = 0
for i in range(12500):
    if(i in ind_dog):
        cc = cc + 1
        continue
    else:
        names_dog_train.append(all_dogs[i])

        cc = 0
for i in range(12500):
    if(i in ind_cat):
        cc = cc + 1
        continue
    else:
        names_cat_train.append(all_cats[i])

all_train_ids = np.concatenate((names_dog_train,names_cat_train))
all_test_ids = np.concatenate((names_dog_test,names_cat_test))
all_train_ids.shape,all_test_ids.shape
all_train_labels = np.concatenate((np.zeros((10000,1)),np.ones((10000,1))))
all_test_labels = np.concatenate((np.zeros((2500,1)),np.ones((2500,1))))


In [4]:
def my_transform(key):
    train_sequence = [
                      transforms.Resize((224,224)),
                      transforms.RandomHorizontalFlip(),
                      transforms.ToTensor()]
    test_sequence = [transforms.Resize((224,224)),
                    transforms.ToTensor()]
    data_transforms = {'train': transforms.Compose(train_sequence),
                       'test': transforms.Compose(test_sequence)}
    return data_transforms[key]

class CatsandDogs(Dataset):
    def __init__(self,path_is,targets,transform):
        self.path_is = path_is
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.path_is)
    
    def __getitem__(self, idx):
        image_path = self.path_is[idx]
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        target = np.int(self.targets[idx])
        return image,target

dats_train = CatsandDogs(all_train_ids,all_train_labels,transform=my_transform(key="train"))
dats_test = CatsandDogs(all_test_ids,all_test_labels,transform=my_transform(key="test"))
BATCH_SIZE = 128
train_dataloader = DataLoader(dats_train, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_dataloader = DataLoader(dats_test, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Torch Device:", device)

Torch Device: cuda:0


In [5]:
torch.manual_seed(756)
model = torchvision.models.resnet18(pretrained=False)
model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.2),
    
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(256,9)
)
model = model.to(device)

In [6]:
def cumLaplaceDistribution(y_pred,mean,standard_deviation,all_qs):
    term1 = ((1-all_qs) * (y_pred - mean))/standard_deviation
    term1.clamp_(max = 0)
    lesser_term = all_qs * torch.exp(term1)
    term2 = (-1.0 * all_qs * (y_pred - mean))/standard_deviation
    term2.clamp_(max = 0)
    greater_term = 1 - ((1-all_qs) * torch.exp(term2))
    dummy_ones = torch.ones_like(mean)
    y_dummy_pred = torch.div(y_pred,dummy_ones)
    y_dummy_pred[y_pred >= mean] = 1.0
    y_dummy_pred[y_pred < mean] = 0.0
    return ((1 - y_dummy_pred) * lesser_term )+  (y_dummy_pred * greater_term)


def logLikelihoodLoss(y_true,y_pred,mean,standard_deviation,all_qs):
    new_pred = y_pred
    prob_tens = cumLaplaceDistribution(0.0,mean = new_pred,standard_deviation = standard_deviation,all_qs = all_qs)
    prob_tens.clamp_(min = 1e-7,max = 1 - 1e-7)
    if_one = y_true * torch.log(1 - prob_tens)
    if_zero = (1 - y_true) * torch.log(prob_tens)
    result = - 1 * torch.mean(if_one + if_zero)
    return result

def customLoss(y_true, y_pred, mean, standard_deviation, all_qs, penalty):
    ind_losses = []
    for i,j in enumerate(all_qs):
        solo_loss = logLikelihoodLoss(y_true[:,0],y_pred[:,i] ,mean, standard_deviation, j)
        ind_losses.append(solo_loss)
    zero = torch.Tensor([0]).to(device)
    dummy1 = y_pred[:,1:] - y_pred[:,:-1]
    dummy2 = penalty * torch.mean(torch.max(zero,-1.0 * dummy1))
    total_loss  = torch.mean(torch.stack(ind_losses)) +dummy2
    return total_loss

def customTestPred(y_pred,mean,standard_deviation,all_qs,batch_size = 1):
    if(batch_size == 1):
        acc = []
        cdfs = []
        eps = 1e-10
        val = (y_pred - mean)/standard_deviation 
        for xx in range(batch_size):
            if(y_pred < mean.item()):
                lesser_term = all_qs * torch.exp((1 - all_qs) * val.item())
                lesser_term  = 1 - lesser_term
                cdfs.append(lesser_term.item())
                if(lesser_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
            
            elif(y_pred >= mean.item()):
                greater_term = 1 - ((1-all_qs) * torch.exp(-1 * all_qs * val.item()))
                greater_term = 1 - greater_term
                cdfs.append(greater_term.item())
                if(greater_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
    
    elif(batch_size > 1):
        acc = []
        cdfs = []
        eps = 1e-10
        val = (y_pred - mean)/standard_deviation 
        for xx in range(batch_size):
            if(y_pred < mean[xx]):
                lesser_term = all_qs * torch.exp((1 - all_qs) * val[xx])
                lesser_term  = 1 - lesser_term
                cdfs.append(lesser_term.item())
                if(lesser_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
            elif(y_pred >= mean[xx]):
                greater_term = 1 - ((1-all_qs) * torch.exp(-1 * all_qs * val[xx]))
                greater_term = 1 - greater_term
                cdfs.append(greater_term.item())
                if(greater_term.item() >= 0.5):
                    acc.append([1])
                else:
                    acc.append([0])
    return torch.Tensor(acc).to(device).reshape(-1,1),torch.Tensor(cdfs).to(device).reshape(-1,1)

def acc_Q(train_preds,train_labels):
    train_preds = np.array(train_preds).reshape(-1,1)
    train_labels = np.array(train_labels).reshape(-1,1)

    cdfs_acc,_ = customTestPred(0,train_preds,standard_deviation = 1,all_qs = torch.Tensor([0.5]),
                                batch_size = train_preds.shape[0])

    count = 0
    for i,j in zip(cdfs_acc,train_labels):
        if(i.item() == j[0]):
            count += 1
    return count/train_labels.shape[0]

def acc_tests(test_preds,test_labels):
    test_preds = np.array(test_preds).reshape(-1,1)
    test_labels = np.array(test_labels).reshape(-1,1)
    cdfs_acc,_ = customTestPred(0,test_preds,standard_deviation = 1,all_qs = torch.Tensor([0.5]),
                                batch_size = test_preds.shape[0])

    count = 0
    for i,j in zip(cdfs_acc,test_labels):
        if(i.item() == j[0]):
            count += 1
    return count/test_labels.shape[0]


In [7]:
def train(model,optimizer,loader,epochs, verbose=True):
    train_preds_Q = []
    train_preds_bce = []
    train_labels = []
    model.train()
    for i,j in enumerate(loader):
        inputs,labels = j[0],j[1]
        inputs = inputs.to(device)
        labels = labels.to(device)
        batch_size = labels.shape
        optimizer.zero_grad()
        op_qs = model(inputs)
        lossQ = customLoss(labels.reshape(-1,1),op_qs, mean_is,std_is,all_qs,penalty)
        lossQ.backward()
        optimizer.step()
        for lag in op_qs[:,4].detach().reshape(-1,1):
            train_preds_Q.append(lag.item())
        for lag in labels.reshape(-1,1):
            train_labels.append(lag.item())
    
    acc_is_Q = acc_Q(train_preds_Q,train_labels)
    if verbose:
        print("[%d/%d] Train Acc Q : %f "%(epochs,total_epochs,acc_is_Q))
    return acc_is_Q

In [8]:
def test(model,loader,epochs,verbose=True):
    model.eval()
    test_preds_Q = []
    test_preds_bce = []
    test_labels = []
    with torch.no_grad():
        for i,j in enumerate(loader):
            inputs,labels = j[0],j[1]
            inputs = inputs.to(device)
            labels = labels.to(device)
            op_qs = model(inputs)
            for lag in op_qs[:,4].detach().reshape(-1,1):
                test_preds_Q.append(lag.item())
            for lag in labels.reshape(-1,1):
                test_labels.append(lag.item())
                
    acc_is_Q = acc_tests(test_preds_Q,test_labels)
    print("[%d/%d] Test Acc Q : %f  "%(epochs,total_epochs,acc_is_Q))




In [9]:
def quantileCDF(x, tau):
    if x>0:
        return 1 - tau*np.exp((tau-1)*x)
    else:
        return (1 - tau)*np.exp(tau*x)

In [10]:
lr_is = 1e-2 
optimizer = torch.optim.Adam(model.parameters(), lr = lr_is)
all_qs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
all_qs = torch.Tensor(all_qs).to(device)
mean_is = 0
std_is = 1
penalty = 1
epsilon = 0.00
total_epochs = 15
verbosity = True

In [11]:
for i in range(total_epochs):
    acc_train = train(model,optimizer, train_dataloader,i+1,verbosity)
    acc_test = test(model,test_dataloader,i+1,verbosity)

[1/15] Train Acc Q : 0.576800 
[1/15] Test Acc Q : 0.593400  
[2/15] Train Acc Q : 0.639800 
[2/15] Test Acc Q : 0.536600  
[3/15] Train Acc Q : 0.704650 
[3/15] Test Acc Q : 0.529400  
[4/15] Train Acc Q : 0.764600 
[4/15] Test Acc Q : 0.661600  
[5/15] Train Acc Q : 0.813150 
[5/15] Test Acc Q : 0.810200  
[6/15] Train Acc Q : 0.846150 
[6/15] Test Acc Q : 0.718600  
[7/15] Train Acc Q : 0.873300 
[7/15] Test Acc Q : 0.770600  
[8/15] Train Acc Q : 0.895800 
[8/15] Test Acc Q : 0.891000  
[9/15] Train Acc Q : 0.917100 
[9/15] Test Acc Q : 0.847200  
[10/15] Train Acc Q : 0.929800 
[10/15] Test Acc Q : 0.873000  
[11/15] Train Acc Q : 0.931800 
[11/15] Test Acc Q : 0.893000  
[12/15] Train Acc Q : 0.945250 
[12/15] Test Acc Q : 0.920200  
[13/15] Train Acc Q : 0.949300 
[13/15] Test Acc Q : 0.922600  
[14/15] Train Acc Q : 0.954800 
[14/15] Test Acc Q : 0.818200  
[15/15] Train Acc Q : 0.960200 
[15/15] Test Acc Q : 0.919000  


In [12]:
delta_total = [0,0,0,0,0]
delta_misc = [0,0,0,0,0]
accept_total = [0,0,0,0,0]
with torch.no_grad():
    all_preds = [[] for i in range(9)]
    test_labels = []
    for i,j in test_dataloader:
        inputs,labels = i.to(device),j.to(device)
        op_qs = model(inputs)
        for itemset in op_qs.detach():
            for quant in range(9):
                all_preds[quant].append(itemset[quant].item())
        for lbl in labels.reshape(-1,1):
            test_labels.append(lbl.item())
    
    for i,j in train_dataloader:
        inputs,labels = i.to(device),j.to(device)
        op_qs = model(inputs)
        for itemset in op_qs.detach():
            for quant in range(9):
                all_preds[quant].append(itemset[quant].item())
        for lbl in labels.reshape(-1,1):
            test_labels.append(lbl.item())

correct_counter = 0
for i in range(len(test_labels)):
    start = 4
    left = start
    right = start
    found = False
    count = 0
    medprob = quantileCDF(all_preds[start][i], 0.5)
    while (left>-1 and not found):
        q_left = all_preds[left][i]
        q_right = all_preds[right][i]
        p_left = quantileCDF(q_left, 0.5)
        p_right = quantileCDF(q_right, 0.5)
        left -=1
        right +=1
        if (q_left < 0.5 and q_right>0.5):
            found = True
        else:
            count +=1
    delta_total[count-1] +=1
    for temp in range(5):
        if count-1>=temp:
            accept_total[temp] +=1
    if (test_labels[i]==0 and medprob<=0.5) or (test_labels[i]==1 and medprob>0.5):
        correct_pred = True
        correct_counter += 1
    else:
        correct_pred = False
    if not correct_pred:
        delta_misc[count-1] +=1


In [13]:
print("Accuracy:{:.2f}".format(correct_counter/len(test_labels)))
print()

print("Delta      |",end=" ")
for i in range(5):
    print("{:.2f}".format(0.1*(i+1)), end=" | ")
print()
print("Misc. Rate |",end=" ")
for i in range(5):
    if delta_total[i] != 0:
        val = delta_misc[i]/delta_total[i]
    else:
        val = 0
    print("{:.2f}".format(val), end=" | ")
print()
print("Ret.  Rate |",end=" ")
for i in accept_total:
    print("{:.2f}".format(i/len(test_labels)), end=" | ")

Accuracy:0.95

Delta      | 0.10 | 0.20 | 0.30 | 0.40 | 0.50 | 
Misc. Rate | 0.40 | 0.30 | 0.18 | 0.09 | 0.01 | 
Ret.  Rate | 1.00 | 0.96 | 0.92 | 0.86 | 0.75 | 